In [1]:
from matplotlib import font_manager, rc  # 한글사용 matplotlib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import csv
import codecs
import numpy as np
import datetime
import json
import tensorflow as tf
import jpype
from konlpy.tag import Okt
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame as df
InteractiveShell.ast_node_interactivity = 'all'
os.chdir("C:/Users/windd/pythondata")
%matplotlib inline
font_name = font_manager.FontProperties(
    fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df_afsnt = pd.read_csv('bigcontest/2019빅콘테스트_퓨처스리그2/AFSNT.csv',engine='python',encoding='cp949')
df_afsnt["STT"] = pd.to_datetime(df_afsnt["STT"], format = "%H:%M").dt.time
df_afsnt["ATT"] = pd.to_datetime(df_afsnt["ATT"], format = "%H:%M").dt.time

df_afsnt = df_afsnt.rename(columns={ 
    "SDT_YY": "year", "SDT_MM":"month", "SDT_DD":"day",  "SDT_DY":"wday", 
    "ARP":"origin", "ODP":"dest", 
    "FLO":"airline", "FLT":"flight" ,"REG":"tailnum", 
    "AOD":"is_arrive", 
    "IRR":"is_regular", 
    "STT":"sched_time", "ATT":"real_time", 
    "DLY":"is_delay", "DRR":"cause_delay", 
    "CNL":"is_cancel", "CNR":"cause_cancel"
})
df_afsnt["date"] = pd.to_datetime(df_afsnt[["year", "month", "day"]])
df_afsnt["sched_datetime"] = pd.to_datetime(df_afsnt['date'].astype(str) + " " + df_afsnt["sched_time"].astype(str))
df_afsnt["real_datetime"] = pd.to_datetime(df_afsnt['date'].astype(str) + " " + df_afsnt["real_time"].astype(str))

# delay 변수 생성
temp = df_afsnt["real_datetime"] - df_afsnt["sched_datetime"] 
temp = pd.DataFrame(temp.dt.components)
df_afsnt["delay"] = temp['minutes'] + 60 * temp['hours'] + 1440 * temp['days']
del temp

df_afsnt.head()

,year,month,day,wday,origin,dest,airline,flight,tailnum,is_arrive,...,sched_time,real_time,is_delay,cause_delay,is_cancel,cause_cancel,date,sched_datetime,real_datetime,delay
0,2017,1,1,일,ARP3,ARP6,J,J1955,SEw3NzE4,D,...,10:05:00,10:32:00,N,NaN,N,NaN,2017-01-01,2017-01-01 10:05:00,2017-01-01 10:32:00,27
1,2017,1,1,일,ARP3,ARP6,J,J1954,SEw3NzE4,A,...,09:30:00,09:31:00,N,NaN,N,NaN,2017-01-01,2017-01-01 09:30:00,2017-01-01 09:31:00,1
2,2017,1,1,일,ARP3,ARP6,J,J1956,SEw3NzE4,A,...,12:45:00,13:03:00,N,NaN,N,NaN,2017-01-01,2017-01-01 12:45:00,2017-01-01 13:03:00,18
3,2017,1,1,일,ARP3,ARP6,J,J1957,SEw3NzE4,D,...,13:25:00,14:09:00,Y,C02,N,NaN,2017-01-01,2017-01-01 13:25:00,2017-01-01 14:09:00,44
4,2017,1,1,일,ARP3,ARP6,J,J1958,SEw3NzE4,A,...,16:10:00,16:31:00,N,NaN,N,NaN,2017-01-01,2017-01-01 16:10:00,2017-01-01 16:31:00,21


In [ ]:
# 파생변수 - 혼잡도

In [3]:
df_afsnt_d = df_afsnt[df_afsnt['is_arrive']=='D']

In [ ]:
# 연습용

test = df_afsnt_d.iloc[:1000]
test = test.sort_values(['origin','date','sched_time'])
test.index = list(range(len(test)))


base_time = datetime.timedelta(minutes=30)
congestion = []

for i in range(len(test)):
    
    airport = test['origin'][i]
    sched_time = test['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time + base_time
    congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(test)-30) else len(test) for x in [i+30]]
    
    
    for j in range(num[0], num2[0],1):
        if (airport == test['origin'][j]) and (lower_time <= test['sched_datetime'][j]) and (test['sched_datetime'][j] <= upper_time):
            congestion_cnt +=1
    congestion.append(congestion_cnt)

In [ ]:
# 구하는 방식은 그대로 사용하면됨
# 다만 전처리하면서 제거되는 행들에 의해서 값이 변하기 때문에
# 나중에 다시 돌려야함

In [59]:
df_afsnt_d = df_afsnt_d.sort_values(['origin','date','sched_time'])
df_afsnt_d.index = list(range(len(df_afsnt_d)))

base_time = datetime.timedelta(minutes=30)
congestion = []

for i in range(len(df_afsnt_d)):
    
    airport = df_afsnt_d['origin'][i]
    sched_time = df_afsnt_d['sched_datetime'][i]
    lower_time = sched_time - base_time
    upper_time = sched_time + base_time
    congestion_cnt = 0
    num = [x if x >= 0 else 0 for x in [i-30]]
    num2 = [x if x <= (len(df_afsnt_d)-30) else len(df_afsnt_d) for x in [i+30]]
    
    for j in range(num[0], num2[0],1):
        if (airport == df_afsnt_d['origin'][j]) and (lower_time <= df_afsnt_d['sched_datetime'][j]) and (df_afsnt_d['sched_datetime'][j] <= upper_time):
            congestion_cnt +=1
    congestion.append(congestion_cnt)
        

In [60]:
df_afsnt_d['congestion'] = congestion

In [ ]:
df_afsnt_d = pd.read_excel('bigcontest/혼잡도포함.xlsx')

In [ ]:
pd.set_option('display.max_rows', 10000)
df_afsnt_d[['origin','date','sched_time','congestion']]

In [138]:
df_afsnt_d[['origin','date','sched_time','congestion']].head(100)

,origin,date,sched_time,congestion
0,ARP1,2017-01-01,06:10:00,8
1,ARP1,2017-01-01,06:15:00,10
2,ARP1,2017-01-01,06:20:00,10
3,ARP1,2017-01-01,06:25:00,10
4,ARP1,2017-01-01,06:30:00,13
5,ARP1,2017-01-01,06:30:00,13
6,ARP1,2017-01-01,06:40:00,14
7,ARP1,2017-01-01,06:40:00,14
8,ARP1,2017-01-01,06:45:00,13
9,ARP1,2017-01-01,06:45:00,13


In [139]:
df_afsnt_d[['origin','date','sched_time','congestion']].tail(100)

,origin,date,sched_time,congestion
493617,ARP9,2019-06-16,17:50:00,1
493618,ARP9,2019-06-16,19:50:00,1
493619,ARP9,2019-06-17,10:20:00,1
493620,ARP9,2019-06-17,11:45:00,1
493621,ARP9,2019-06-17,14:05:00,1
493622,ARP9,2019-06-17,15:00:00,1
493623,ARP9,2019-06-17,16:00:00,1
493624,ARP9,2019-06-17,17:50:00,1
493625,ARP9,2019-06-17,18:50:00,1
493626,ARP9,2019-06-18,10:20:00,1


In [65]:
df_afsnt_d.congestion.value_counts().sum()
len(df_afsnt_d)

493717

493717

In [66]:
df_afsnt_d.congestion.value_counts()

1     36253
16    35990
15    35513
14    33937
17    33628
13    32455
2     31383
12    29793
18    29736
11    25075
19    24150
3     18218
5     18078
20    16908
4     16601
10    15836
6     13268
9     11906
21    10160
7      8350
8      8014
22     4458
23     1677
24      950
25      750
26      502
27       84
28       16
29       13
31       11
32        2
33        2
Name: congestion, dtype: int64

In [ ]:
# 파생변수 - 링크 데이터
# 단순 링크가 아닌 고유 등록기호에만 영향을 생각할 수 있을까?

In [4]:
df_afsnt_sorted = df_afsnt.sort_values(['date','tailnum','sched_datetime'])
df_afsnt_sorted_d = df_afsnt_sorted[df_afsnt_sorted['is_arrive']=='D']

In [5]:
df_afsnt_sorted_d = df_afsnt_sorted_d[df_afsnt_sorted_d['is_cancel']!='Y']
df_afsnt_sorted_d = df_afsnt_sorted_d[~((df_afsnt_sorted_d['origin']=='ARP1') & (df_afsnt_sorted_d['dest']=='ARP1'))]
df_afsnt_sorted_d = df_afsnt_sorted_d[~(df_afsnt_sorted_d['tailnum'].isnull())]

In [6]:
df_afsnt_sorted_d.index = list(range(len(df_afsnt_sorted_d)))
len(df_afsnt_sorted_d)

489575

In [7]:
df_afsnt_sorted_d.tail(10)[['date','origin','dest','tailnum','sched_datetime','is_arrive']]

,date,origin,dest,tailnum,sched_datetime,is_arrive
489565,2019-06-30,ARP3,ARP2,SEw4MzY1,2019-06-30 18:35:00,D
489566,2019-06-30,ARP2,ARP1,SEw4MzY1,2019-06-30 21:00:00,D
489567,2019-06-30,ARP4,ARP3,SEw4MzYz,2019-06-30 06:20:00,D
489568,2019-06-30,ARP3,ARP1,SEw4MzYz,2019-06-30 07:55:00,D
489569,2019-06-30,ARP1,ARP3,SEw4MzYz,2019-06-30 09:45:00,D
489570,2019-06-30,ARP3,ARP1,SEw4MzYz,2019-06-30 11:35:00,D
489571,2019-06-30,ARP1,ARP3,SEw4MzYz,2019-06-30 13:20:00,D
489572,2019-06-30,ARP3,ARP1,SEw4MzYz,2019-06-30 15:05:00,D
489573,2019-06-30,ARP1,ARP3,SEw4MzYz,2019-06-30 17:25:00,D
489574,2019-06-30,ARP3,ARP4,SEw4MzYz,2019-06-30 19:10:00,D


In [8]:
link_ref = df_afsnt_sorted_d.pivot_table(index = ['date','tailnum'],aggfunc='count',values='is_arrive' )
link_ref.head(100)

is_arrive
date       tailnum            
2017-01-01 SEw3MjQ3          8
           SEw3NDYw          6
           SEw3NTI1          6
           SEw3NTI2          1
           SEw3NTQw          4
           SEw3NTU3          8
           SEw3NTU4          8
           SEw3NTY0          8
           SEw3NTY2          8
           SEw3NTY4          8
           SEw3NTY5          8
           SEw3NTYw          2
           SEw3NTYz          9
           SEw3NTk0          9
           SEw3NTk5          4
           SEw3NzA0          1
           SEw3NzA1          4
           SEw3NzA2          7
           SEw3NzA4          7
           SEw3NzAz         10
           SEw3NzE2          8
           SEw3NzE4          8
           SEw3NzE5          8
           SEw3NzEz          2
           SEw3NzI0          4
           SEw3NzI1          4
           SEw3NzI2          1
           SEw3NzI3          7
           SEw3NzIy          2
           SEw3NzIz          7
...                        ...
           SEw4MjA1          8
           SEw4MjA3          8
           SEw4MjEz          6
           SEw4MjI0          2
           SEw4MjIx          1
           SEw4MjM1          4
           SEw4MjM2          8
           SEw4MjM3         10
           SEw4MjM5          5
           SEw4MjQ0          1
           SEw4MjQ2          1
           SEw4MjQ3          4
           SEw4MjQw          1
           SEw4MjQx          6
           SEw4MjU2          2
           SEw4MjUz          8
           SEw4MjY0          8
           SEw4MjY4          7
           SEw4MjYx          6
           SEw4Mjc3          1
           SEw4Mjc4          1
           SEw4Mjg5         10
           SEw4Mjk3          9
           SEw4Mjky          4
2017-01-02 SEw3MjQ3          8
           SEw3NDYw          1
           SEw3NTI1          6
           SEw3NTU3          8
           SEw3NTU4          6
           SEw3NTY0          8

[100 rows x 1 columns]

In [10]:
link_ref_list = list(link_ref['is_arrive'].values)

In [11]:
# 링크의 경우 출발기준으로 진행하고 있는데
# 도착인 경우에는 어떻게 접근?
# 그냥 출도착 쌍에 맞게 진행?

link = []

for i in link_ref_list:
    link_range = list(range(i))
    link.append(link_range)
    
link = sum(link, [])
df_afsnt_sorted_d['link'] = link

In [12]:
df_afsnt_sorted_d[['date','origin','dest','tailnum','sched_datetime','link']].head(100)

,date,origin,dest,tailnum,sched_datetime,link
0,2017-01-01,ARP1,ARP3,SEw3MjQ3,2017-01-01 07:20:00,0
1,2017-01-01,ARP3,ARP1,SEw3MjQ3,2017-01-01 09:15:00,1
2,2017-01-01,ARP1,ARP3,SEw3MjQ3,2017-01-01 11:05:00,2
3,2017-01-01,ARP3,ARP1,SEw3MjQ3,2017-01-01 13:00:00,3
4,2017-01-01,ARP1,ARP3,SEw3MjQ3,2017-01-01 14:50:00,4
5,2017-01-01,ARP3,ARP1,SEw3MjQ3,2017-01-01 16:45:00,5
6,2017-01-01,ARP1,ARP3,SEw3MjQ3,2017-01-01 18:35:00,6
7,2017-01-01,ARP3,ARP1,SEw3MjQ3,2017-01-01 20:40:00,7
8,2017-01-01,ARP1,ARP3,SEw3NDYw,2017-01-01 09:55:00,0
9,2017-01-01,ARP3,ARP1,SEw3NDYw,2017-01-01 12:10:00,1


In [14]:
link_table = df_afsnt_sorted_d.pivot_table(index = ['origin','dest','link'], aggfunc='count',values='is_arrive')
link_table = link_table.reset_index(['origin','dest','link'])
link_table.sort_values(['origin','dest']).head(50)

,origin,dest,link,is_arrive
0,ARP1,ARP10,0,1
1,ARP1,ARP10,2,2
2,ARP1,ARP11,0,597
3,ARP1,ARP11,1,288
4,ARP1,ARP11,2,41
5,ARP1,ARP11,3,69
6,ARP1,ARP11,4,466
7,ARP1,ARP11,5,189
8,ARP1,ARP11,6,58
9,ARP1,ARP12,0,874


In [20]:
# 공항 출발 time sequence 생성

df_afsnt_sorted_d2 = df_afsnt_sorted_d.sort_values(['date','origin','sched_datetime','real_datetime'])
df_afsnt_sorted_d2.index = list(range(len(df_afsnt_sorted_d2)))
df_afsnt_sorted_d2.head(100)

,year,month,day,wday,origin,dest,airline,flight,tailnum,is_arrive,...,real_time,is_delay,cause_delay,is_cancel,cause_cancel,date,sched_datetime,real_datetime,delay,link
0,2017,1,1,일,ARP1,ARP3,A,A1901,SEw3Nzc2,D,...,06:18:00,N,NaN,N,NaN,2017-01-01,2017-01-01 06:10:00,2017-01-01 06:18:00,8,0
1,2017,1,1,일,ARP1,ARP3,A,A1905,SEw4MjM2,D,...,06:25:00,N,NaN,N,NaN,2017-01-01,2017-01-01 06:15:00,2017-01-01 06:25:00,10,0
2,2017,1,1,일,ARP1,ARP3,L,L1751,SEw4MjM3,D,...,06:30:00,N,NaN,N,NaN,2017-01-01,2017-01-01 06:20:00,2017-01-01 06:30:00,10,0
3,2017,1,1,일,ARP1,ARP3,F,F1201,SEw4MjA3,D,...,06:34:00,N,NaN,N,NaN,2017-01-01,2017-01-01 06:25:00,2017-01-01 06:34:00,9,0
4,2017,1,1,일,ARP1,ARP3,H,H1101,SEw4MDMx,D,...,06:38:00,N,NaN,N,NaN,2017-01-01,2017-01-01 06:30:00,2017-01-01 06:38:00,8,0
5,2017,1,1,일,ARP1,ARP3,I,I1301,SEw3NTYz,D,...,06:44:00,N,NaN,N,NaN,2017-01-01,2017-01-01 06:30:00,2017-01-01 06:44:00,14,0
6,2017,1,1,일,ARP1,ARP3,I,I1303,SEw4MDE1,D,...,06:53:00,N,NaN,N,NaN,2017-01-01,2017-01-01 06:40:00,2017-01-01 06:53:00,13,0
7,2017,1,1,일,ARP1,ARP3,A,A1907,SEw3NTk0,D,...,07:54:00,Y,C01,N,NaN,2017-01-01,2017-01-01 06:40:00,2017-01-01 07:54:00,74,0
8,2017,1,1,일,ARP1,ARP3,A,A1903,SEw3Nzcz,D,...,06:55:00,N,NaN,N,NaN,2017-01-01,2017-01-01 06:45:00,2017-01-01 06:55:00,10,0
9,2017,1,1,일,ARP1,ARP3,J,J1204,SEw3Nzg1,D,...,07:02:00,N,NaN,N,NaN,2017-01-01,2017-01-01 06:45:00,2017-01-01 07:02:00,17,0


In [28]:
sequence_ref = df_afsnt_sorted_d2.pivot_table(index = ['date','origin'], aggfunc = 'count', values = 'is_arrive')
sequence_ref_list = list(sequence_ref['is_arrive'].values)

In [29]:
# 링크의 경우 출발기준으로 진행하고 있는데
# 도착인 경우에는 어떻게 접근?
# 그냥 출도착 쌍에 맞게 진행?

sequence = []

for i in sequence_ref_list:
    sequence_range = list(range(i))
    sequence.append(sequence_range)
    
sequence = sum(sequence, [])
df_afsnt_sorted_d2['air_sequence'] = sequence

In [ ]:
df_afsnt_sorted_d2[['date','origin','sched_datetime','air_sequence']]

In [ ]:
# 공항 도착 time sequence 생성